In [1]:
curr_dir = '/user_data/vayzenbe/GitHub_Repos/ginn'

import sys
sys.path.insert(1, f'{curr_dir}/Models')
import os, argparse
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageOps,  ImageFilter
from itertools import chain
import pandas as pd
import numpy as np
import seaborn as sns
import cornet
import model_funcs
import matplotlib.pyplot as plt
from statistics import mean
%matplotlib inline
import LoadFrames
from scipy import stats, spatial

In [59]:
im_dir = f"{curr_dir}/Stim/gray_exp_stim/"
weights_dir = f"/lab_data/behrmannlab/vlad/ginn/model_weights"
results_dir = f"{curr_dir}/Results/fmri_ts"

stim_data = pd.read_csv(f"{curr_dir}/analysis/stim_data.csv")
stim_data.sort_values(by=['stim1', 'stim2'], ascending=True).reset_index()

'''
set model params
'''
model_type = 'classify'
train_cond = ['random','imagenet_objects','imagenet_oneface','mixed_imagenet_vggface','vggface']
model_cond = ['random','general','one_face', 'mixed','face']
n_classes = [600,600, 601, 1200, 600]
epoch = [1,5, 10, 15, 20, 25, 30]
layer =['aIT','pIT'] #set in descending order
#layer = 'aIT'

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

im_count = 15



In [38]:
def extract_acts(model, im):
    """
    Extracts the activations for a series of images
    """
    model.eval()

    with torch.no_grad():

        im = im.cuda()
        output = model(im)
        output =output.view(output.size(0), -1)
        

    return output, label

In [39]:
def create_cos_rdm(output, label):
    """calculate cos similarity between each image"""
    rdm =[]
    im_pairs = []
    for ii in range(0,len(output)):
        for kk in range(ii+1,len(output)):
            sim = cos(output[ii], output[kk])
            sim =sim.cpu().numpy()
            rdm.append(sim)
            im_pairs.append([label[ii], label[kk]])

    return rdm, im_pairs

In [57]:
'''
Reload best ims to calculate final correlation and generate figures
'''

stim_data = pd.read_csv(f"{curr_dir}/analysis/stim_data.csv")
stim_data.sort_values(by=['stim1', 'stim2'], ascending=True).reset_index()
dataset = LoadFrames.LoadFrames(im_dir,  transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=im_count, shuffle=False, drop_last = False,num_workers = 4, pin_memory=True)

for tcn, tc in enumerate(train_cond):
    for ee in epoch:
        for ll in layer:
            model_df = pd.DataFrame(columns = ['stim1','stim2','similarity'])
            if tc == 'random':
                model = model_funcs.load_model(model_type, tc, 0, weights_dir, n_classes[tcn])
                model =  model_funcs.remove_layer(model, ll)

                if ee > 1:
                    break

            else:
                #load first model
                model = model_funcs.load_model(model_type, tc, ee, weights_dir, n_classes[tcn])
                model =  model_funcs.remove_layer(model, ll)
            
            im, label = next(iter(loader))

            out, label = extract_acts(model,im)


            rdm, im_pairs = create_cos_rdm(out, label)

            im_pairs = np.array(im_pairs)
            rdm = np.array(rdm)

            model_df['stim1'] = im_pairs[:,0]
            model_df['stim2'] = im_pairs[:,1]
            model_df['similarity'] = rdm

            model_df = model_df.sort_values(by=['stim1', 'stim2'], ascending=True).reset_index()
            stim_data[f'{model_cond[tcn]}_{ll}_{ee}'] = model_df['similarity']
            


In [58]:
stim_data.to_csv('stim_data.csv',index=False)

In [53]:
tc

'imagenet_objects'